Парсер docdoc.ru.

Часть 2, собирает со страниц клиник данные:
- описание
- специализацию
- кол-во врачей
- основные данные о врачах

Данные записываю сразу в два датасета, в общий, по клиникам, и по врачам.

Последние правки скрипта: 2022-01-06

## Библиотеки и настройки ##

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd
import math

# регулярки
import re

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# путь к файлам

file_path = 'C:/00_Data/stom/'
proxy_list_path = 'C:/work/proxy_list.txt' # путь и названия со списком прокси

In [ ]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]
        
print('Рабочих прокси — ' + str(len(proxy_list)))

In [ ]:
# номер прокси

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

In [ ]:
# запускаю драйвер, проверяю настройки

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

## Загружаю ссылки ##

In [ ]:
# нужный город латиницей, транслитом как на сайте

need_city = input()

In [ ]:
# ссылка на начальную страницу поиска клиник

work_links = pd.read_excel(file_path + f'doc_doc_urls_{need_city}.xlsx')

In [ ]:
work_links.shape

In [ ]:
work_links.sample(2)

In [ ]:
# список ссылок

page_list = work_links['clinic_page'].to_list()

### Код парсера ###

In [ ]:
data = [] # список для данных
data_doc = [] # список для данных о врачах

for page_url in tqdm(page_list): # цикл по списку ссылок
    
    driver.get(page_url)        
    sleep(3)
    
    try: # принимаю куки
        driver.find_element_by_tag_name('button[data-test-id="cookies-banner-button"]').click()
        
    except:
        pass    
    
    # описание клиники
    
    try: # раскрываю "Показать ещё"
        (driver.find_element_by_tag_name('div[itemprop="description"]')
         .find_element_by_tag_name('button')).click()
        
    except:
        pass
    
    try:
        description = (driver.find_element_by_tag_name('div[itemprop="description"]')
                       .text.replace('\n', ''))
        
    except:
        description = 'none'
        
    try:
        specialization = (driver.find_element_by_tag_name('div[data-test-id="clinic-page-info-block__specialities"]')
                          .text.replace('Специализация\n', '').split())
        
    except:
        specialization = 'none'
        
        
    # врачи
    
    # количество врачей
    doc_number = int(re.sub(r'\D', '', driver.find_element_by_class_name('async-cards__title').text))
    
    
    while True: # раскрываю все блоки врачей
        try:
            driver.find_element_by_tag_name('button[data-test-id="async-clinic-card-list__button"]').click()
            sleep(1)
        
        except:
            break    
    
    # краткие данные по врачам
    
    # достаю блоки с врачами    
    doc_blocks = driver.find_elements_by_tag_name('div[data-test-id="doctor-list-page-card"]')
    
    for i in doc_blocks:
        
        # рейтинг
        doc_rating = int(re.sub(r'\D', '',
                                i.find_element_by_class_name('the-rating__mono-fill')
                                .get_attribute('style')))
        
        # количество отзывов и рекомендаций
        try:
            doc_rec = (i.find_element_by_tag_name('div[data-test-id="doctor-list-page-card-rating__recommendations"]')
                       .text.split('\n'))
            
        except:
            doc_rec = 'none'
        
        # имя врача
        doc_name = (i.find_element_by_tag_name('div[data-test-id="doctor-list-page-card__details-desktop"')
                    .find_element_by_tag_name('div[data-test-id="doctor-list-page-card-details__doctor-name"]')
                    .text)
        
        # специализация врача
        doc_specialty = (i.find_element_by_tag_name('div[data-test-id="doctor-list-page-card__details-desktop"')
                         .find_element_by_tag_name('div[data-test-id="doctor-list-page-card-details__specialities"]')
                         .text)
        
        # стаж
        try:
            doc_experience = int(re.sub(r'\D', '',
                                        (i.find_element_by_tag_name
                                         ('div[data-test-id="doctor-list-page-card__details-desktop"')
                                         .find_element_by_tag_name
                                         ('span[data-test-id="doctor-list-page-card-details__experience"]')
                                         .text)))
            
        except:
            doc_experience = 'none'
        
        
        # страница врача
        doc_page = (i.find_element_by_tag_name('div[data-test-id="doctor-list-page-card__details-desktop"')
                    .find_element_by_tag_name('a').get_attribute('href'))
        
        # весь текст блока
        all_text = i.text.split('\n')
        
        
        data_doc.append([doc_name, doc_specialty, doc_experience,
                         doc_rating, doc_rec,
                         all_text,
                         doc_page, page_url])
                
    
    
    data.append([page_url,
                 description, specialization,
                 doc_number])

In [ ]:
# Записываю в датафрейм

head_list = ['clinic_page', 'description', 'specialization', 'doc_number'] # клиники

head_list_doc = ['doc_name', 'doc_specialty', 'doc_experience',
                 'doc_rating', 'doc_rec',
                 'all_text', 'doc_page', 'clinic_page'] # врачи

work = pd.DataFrame(data, columns=head_list)
work_doc = pd.DataFrame(data_doc, columns=head_list_doc)

In [ ]:
work.shape

In [ ]:
work_doc.shape

In [ ]:
# добавляю во фреймы данные из work.links

work = work.merge(work_links, how='left', on='clinic_page')
work_doc = work_doc.merge(work_links, how='left', on='clinic_page')

In [ ]:
# меняю столбцы местами

work = work[['main_clinic', 'actual',
         'clinic_type', 'speciality', 'specialization',
         'doc_number',
         'branch_clinic', 'branch_number',
         'address', 'metro_stations',
         'phone', 'work_hours', 
         'rating', 'rewiev_number', 'branch_rating', 'branch_rewiev_number',
         'description', 'clinic_page'
        ]]

# для датафрейма врачей выкидываю ненужное

work_doc = work_doc[['doc_name', 'doc_specialty', 'doc_experience',
             'doc_rating', 'doc_rec',
             'all_text', 'doc_page',
             'main_clinic', 'clinic_page', 'actual']]

In [ ]:
# записываю в файлы

work.to_excel(file_path + f'doc_doc_data_{need_city}.xlsx', encoding='utf8', index=False)
work_doc.to_excel(file_path + f'doc_doc_data_docs_{need_city}.xlsx', encoding='utf8', index=False)

In [ ]:
driver.quit()